# Yield Curve Analysis Demo

This notebook demonstrates the complete workflow for:
1. Loading U.S. Treasury yields from FRED
2. Fitting Cubic Spline and Nelson–Siegel–Svensson models
3. Visualizing the curves
4. Calculating curve metrics
5. Computing forward rates



In [ ]:
# Import the package
import sys
import os

# Verify we're using the correct Python (should be from venv)
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}\n")

# Check if scipy is available (critical for this project)
try:
    import scipy
    print(f"✓ scipy version: {scipy.__version__}")
except ImportError:
    print("⚠ ERROR: scipy not found!")
    print("  Please select the 'Python (yieldcurve-venv)' kernel:")
    print("  Kernel → Change Kernel → Python (yieldcurve-venv)")
    raise

# Option 1: Use absolute path (most reliable)
PROJECT_ROOT = '/Users/aryansinha/Desktop/ALGORITHMIC TRADING PROJECTS/Interestrate'

# Option 2: Try to find it relative to current directory
# Go up two levels from examples/ directory
current_dir = os.getcwd()
if 'examples' in current_dir or not os.path.exists(os.path.join(PROJECT_ROOT, 'yieldcurve')):
    # Try relative path: go up two levels from current directory
    potential_root = os.path.abspath(os.path.join(current_dir, '..', '..'))
    if os.path.exists(os.path.join(potential_root, 'yieldcurve')):
        PROJECT_ROOT = potential_root

# Add project root to Python path
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Verify
yieldcurve_path = os.path.join(PROJECT_ROOT, 'yieldcurve')
print(f"\n✓ Project root: {PROJECT_ROOT}")
print(f"✓ Current directory: {current_dir}")
print(f"✓ yieldcurve exists: {os.path.exists(yieldcurve_path)}")

if not os.path.exists(yieldcurve_path):
    print(f"\n⚠ WARNING: yieldcurve directory not found!")
    print(f"  Expected at: {yieldcurve_path}")
    print(f"  Please ensure the project structure is correct.")

# Now import
from yieldcurve import (
    get_yield_curve,
    fit_spline,
    fit_nss,
    plot_yield_curves
)
from yieldcurve.utils.metrics import (
    calculate_slope,
    calculate_curvature,
    calculate_forward_rates,
    duration_approx
)
import numpy as np
import matplotlib.pyplot as plt

print("\n✓ All imports successful!")


## Step 1: Load Yields from FRED


In [ ]:
# Fetch current U.S. Treasury yield curve
curve = get_yield_curve()

print("📊 Current U.S. Treasury Yield Curve:")
print("=" * 50)
for maturity, yield_val in curve.items():
    print(f"  {maturity:>4s}: {yield_val:6.2f}%")
print("=" * 50)


## Step 2: Fit Cubic Spline Model


In [ ]:
# Fit cubic spline to the yield curve
spline_model = fit_spline(curve)

print("✅ Cubic Spline Model Fitted")
spline_model.summary()


## Step 3: Fit Nelson–Siegel–Svensson Model


In [ ]:
# Fit NSS model to the yield curve
nss_model = fit_nss(curve)

print("✅ NSS Model Fitted")
nss_model.summary()


## Step 4: Plot Both Curves


In [ ]:
# Plot observed points, spline, and NSS models
plot_yield_curves(
    curve,
    spline_model=spline_model,
    nss_model=nss_model,
    title="U.S. Treasury Yield Curve: Spline vs NSS Models",
    show=True
)


## Step 5: Calculate Curve Metrics


In [ ]:
# Calculate slope (10Y - 2Y)
slope = calculate_slope(curve, short_maturity="2Y", long_maturity="10Y")
print(f"📈 Yield Curve Slope (10Y - 2Y): {slope:.2f} basis points")

# Calculate curvature
curvature = calculate_curvature(curve, short="2Y", medium="5Y", long="10Y")
print(f"📊 Yield Curve Curvature: {curvature:.2f} basis points")
if curvature > 0:
    print("   → Humped curve (belly higher than ends)")
else:
    print("   → Bowed curve (belly lower than ends)")


## Step 6: Compute Forward Rates


In [ ]:
# Generate forward rates from the NSS model
maturity_grid = np.linspace(0.5, 30, 100)
forward_rates = calculate_forward_rates(nss_model, maturity_grid)

# Plot forward curve
plt.figure(figsize=(12, 7))
plt.plot(maturity_grid, forward_rates, label="Forward Rates (NSS)", linewidth=2.5, color="green")
plt.plot(maturity_grid, nss_model(maturity_grid), label="Spot Rates (NSS)", 
         linewidth=2.5, linestyle="--", color="red", alpha=0.7)
plt.xlabel("Maturity (Years)", fontsize=14)
plt.ylabel("Rate (%)", fontsize=14)
plt.title("Forward Rate Curve vs Spot Rate Curve", fontsize=16, fontweight="bold")
plt.grid(True, alpha=0.3)
plt.legend(fontsize=13)
plt.tight_layout()
plt.show()

print(f"📉 1-Year Forward Rate at 5Y: {forward_rates[np.argmin(np.abs(maturity_grid - 5.0))]:.2f}%")
print(f"📉 1-Year Forward Rate at 10Y: {forward_rates[np.argmin(np.abs(maturity_grid - 10.0))]:.2f}%")


## Step 7: Duration Approximation

Calculate modified duration for various maturities.


In [ ]:
# Calculate duration for various maturities
maturities = [1, 2, 5, 10, 20, 30]
print("📏 Modified Duration (Years):")
print("=" * 40)
for t in maturities:
    dur = duration_approx(nss_model, t)
    print(f"  {t:2d}Y: {dur:6.2f} years")
print("=" * 40)
